In [30]:
import os
import pandas as pd
import numpy as np

In [31]:
df=pd.read_csv("../data_given/insurance.csv")

In [32]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')

In [33]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [12]:
df.describe().loc[['max','min']].to_json('schema_in.json')

In [13]:
class NotInRange(Exception):
    def __init__(self,message='Value not in range'):
        self.message=message
        super().__init__(self.message)
    

In [14]:
raise NotInRange

NotInRange: Value not in range

In [15]:
df.columns


Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')

In [19]:
df.describe().loc[['max','min']]

,age,bmi,children,charges
max,64.0,53.13,5.0,63770.42801
min,18.0,15.96,0.0,1121.87390


In [21]:
df['smoker'].value_counts()

no     1064
yes     274
Name: smoker, dtype: int64

In [34]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')

In [35]:
df['sex'].value_counts()

male      676
female    662
Name: sex, dtype: int64

In [54]:
import category_encoders as ce
encoder= ce.OrdinalEncoder(cols=cat_cols,return_df=True
                           )

In [80]:
cat_cols=df.select_dtypes(include='object').columns
num_cols=df.select_dtypes(exclude='object').columns

In [81]:
num_cols

Index(['age', 'bmi', 'children', 'charges'], dtype='object')

In [83]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
trf=ColumnTransformer([
    ('encoder',encoder,cat_cols),
    ('std',StandardScaler(),num_cols[:-1])
],remainder='passthrough')

In [84]:
x=df.drop(columns='charges')
x.head()

,age,sex,bmi,children,smoker,region
0,19,female,27.900,0,yes,southwest
1,18,male,33.770,1,no,southeast
2,28,male,33.000,3,no,southeast
3,33,male,22.705,0,no,northwest
4,32,male,28.880,0,no,northwest


In [93]:
x_trf=trf.fit_transform(x)

In [94]:
x_trf.shape

(1338, 6)

In [99]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
x_train,x_test,y_train,y_test=train_test_split(x_trf,df['charges'],test_size=0.25,random_state=42)

In [96]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()
rf.fit(x_train,y_train)

RandomForestRegressor()

In [98]:
y_pred=rf.predict(x_test)

In [100]:
r2_score(y_test,y_pred)

0.8529065986381547